# Daily Client Report Python script


## Report Header: 

Date and Time 

Client Name 

Report Contact Information (Support email or dedicated client manager contact) 









## Key Metrics (Critical for Client Operations): 

### Payments Processed Successfully: 

*Number of successful payments processed (e.g., 150 transactions).* 

### Payments Pending or Failed: 

*Number of pending payments.* 

*Number of failed payments, with a brief summary of reasons (e.g., insufficient funds, incorrect details).* 

### System Availability: 

*Uptime percentage for the past 24 hours.* 

*Any system interruptions or planned downtimes.* 

### Processing Times: 

*Average processing time for payments.* 

*Peak processing time during the day.* 

### Error or Exception Summary: 

*Summary of critical errors (if any) affecting payment or system operations.* 

*Status and resolution progress (if applicable).* 

### Bank Connectivity Status: 

*Status of key bank integrations (e.g., 100% operational, delays reported with XYZ Bank).* 

### Security Alerts: 

*Any detected anomalies or security incidents (if applicable).* 

### Insights and Recommendations: 

*Trends or anomalies noticed in transaction volumes or processing times.* 

*Suggested actions (e.g., "Please verify pending payment details to ensure timely processing.").* 

### Upcoming Planned Activities: 

*Scheduled maintenance or updates.* 

*New features being rolled out that could affect operations.*

### Contact for Immediate Issues: 

*Dedicated support channel (e.g., "Please contact support@starfish.digital for urgent matters.").* 

## Packages and Libraries

In [ ]:
!pip install pymongo
!pip install matplotlib 
!pip install requests
!pip install schedule
!pip install time
!pip install fpdf
!pip install reportlab 
!pip install smtplib 
!pip install jira 


In [ ]:
from pymongo import MongoClient #This package will help us to get access to the MongoDB connection link
import pandas as pd #Set up a dataframe 
from datetime import datetime, timedelta # Set up the time and date



## Connect to MongoDB

We are going to use the link connection/MongoDB, specify the data base and thte collection where we want to fetch the data. In our case 

**MongDB_link** = "Your link there"

**DataBase**= "Databasename"

**Collection**= "payment-request"


We use : **"ACCP", "ACSp", "RCVD"** to describe a successful payment. Those data can be found in the ISOStatus of the payment-request.Let's extract them and count them.


We use : **"PDNG", "RJCT"** to describe a successful payment. Those data can be found in the ISOStatus of the payment-request.Let's extract them and count them.


## Generate the report

## Now let's do the same exercise but for the last 24 Hours

### successful transactions in last 24 hours

Query : 


{
  "$and": [
    {
      "$or": [
        { "paymentDetails.createdDate": { "$gte": new Date(new Date().getTime() - 24*60*60*1000) } },
        { "completionDate": { "$gte": new Date(new Date().getTime() - 24*60*60*1000) } }
      ]
    },
    { "paymentDetails.ISOStatus": { "$in": ["ACCP", "ACSP"] } }
  ]
}

In [ ]:
MONGO_URI = "xxxxxxxxxxx"

DATABASE_NAME = "xxxxxxxxxxx"  
COLLECTION_NAME = "payment-request"  

# Connect to MongoDB
client = MongoClient(MONGO_URI)
db = client[DATABASE_NAME]
collection = db[COLLECTION_NAME]

# Calculate the last 24 hours range
end_date = datetime.utcnow()
start_date = end_date - timedelta(hours=24)

# Query to count successful payments in the last 24 hours
query_succ = {
    "paymentDetails.ISOStatus": {"$in": ["ACCP", "ACSp","ACSC"]},
    "paymentDetails.createdDate": {"$gte": start_date, "$lt": end_date}
}
successful_payments_count = collection.count_documents(query_succ)
print(f"Number of Successful Payments in the Last 24 Hours: {successful_payments_count}")

# Aggregation pipeline for total amount of successful payments
pipeline_succ = [
    {"$unwind": "$paymentDetails"},  # Flatten the paymentDetails array
    {"$match": {
        "paymentDetails.ISOStatus": {"$in": ["ACCP", "ACSp","ACSC"]},  # Filter for successful payments
        "paymentDetails.createdDate": {"$gte": start_date, "$lt": end_date},  # Filter by last 24 hours
        "paymentDetails.amount": {"$exists": True, "$ne": None}  # Ensure amount exists
    }},
    {"$addFields": {"paymentDetails.amount": {"$toDouble": "$paymentDetails.amount"}}},  # Convert amount to double
    {"$group": {"_id": None, "totalAmount": {"$sum": "$paymentDetails.amount"}}}  # Sum the amounts
]

# Execute the aggregation
result_succ = list(collection.aggregate(pipeline_succ))
total_amount_succ = result_succ[0]["totalAmount"] if result_succ else 0

print(f"Total Amount Processed for Successful Payments in the Last 24 Hours: {total_amount_succ}")

## Pending transactions in last 24 hours

Query={
  "$and": [
    {
      "$or": [
        { "paymentDetails.createdDate": { "$gte": new Date(new Date().getTime() - 24*60*60*1000) } },
        { "completionDate": { "$gte": new Date(new Date().getTime() - 24*60*60*1000) } }
      ]
    },
    { "paymentDetails.ISOStatus": { "$in": ["PDNG", "RCVD"] } }
  ]
}

In [ ]:
  

# Connect to MongoDB
client = MongoClient(MONGO_URI)
db = client[DATABASE_NAME]
collection = db[COLLECTION_NAME]

# Calculate the last 24 hours range
end_date = datetime.utcnow()
start_date = end_date - timedelta(hours=24)

# Query to count successful payments in the last 24 hours
query_pending = {
    "paymentDetails.ISOStatus": {"$in": ["PDNG", "RCVD"]},
    "paymentDetails.createdDate": {"$gte": start_date, "$lt": end_date}
}
pending_payments_count = collection.count_documents(query_pending)
print(f"Number of Pending Payments in the Last 24 Hours: {pending_payments_count}")

# Aggregation pipeline for total amount of successful payments
pipeline_pending = [
    {"$unwind": "$paymentDetails"},  # Flatten the paymentDetails array
    {"$match": {
        "paymentDetails.ISOStatus": {"$in": ["PDNG", "RCVD"]},  # Filter for successful payments
        "paymentDetails.createdDate": {"$gte": start_date, "$lt": end_date},  # Filter by last 24 hours
        "paymentDetails.amount": {"$exists": True, "$ne": None}  # Ensure amount exists
    }},
    {"$addFields": {"paymentDetails.amount": {"$toDouble": "$paymentDetails.amount"}}},  # Convert amount to double
    {"$group": {"_id": None, "totalAmount": {"$sum": "$paymentDetails.amount"}}}  # Sum the amounts
]

# Execute the aggregation
result_pending = list(collection.aggregate(pipeline_pending))
total_amount_pending = result_pending[0]["totalAmount"] if result_pending else 0

print(f"Total Amount Processed for pending Payments in the Last 24 Hours: {total_amount_pending}")

## Failed transactions in last 24 hours

Query= {
  "$and": [
    {
      "$or": [
        { "paymentDetails.createdDate": { "$gte": new Date(new Date().getTime() - 24*60*60*1000) } },
        { "completionDate": { "$gte": new Date(new Date().getTime() - 24*60*60*1000) } }
      ]
    },
    { "paymentDetails.ISOStatus": { "$in": ["RJCT"] } }
  ]
}

In [ ]:


# Connect to MongoDB
client = MongoClient(MONGO_URI)
db = client[DATABASE_NAME]
collection = db[COLLECTION_NAME]

# Calculate the last 24 hours range
end_date = datetime.utcnow()
start_date = end_date - timedelta(hours=24)


query_2 = {
    "paymentDetails.ISOStatus": {"$in": ["RJCT"]},
    "paymentDetails.createdDate": {"$gte": start_date, "$lt": end_date}
}
fail_payments_count = collection.count_documents(query_2)
print(f"Number of Failed Payments in the Last 24 Hours: {fail_payments_count}")

# Aggregation pipeline for total amount of failed payments
pipeline_fail = [
    {"$unwind": "$paymentDetails"},  # Flatten the paymentDetails array
    {"$match": {
        "paymentDetails.ISOStatus": {"$in": ["RJCT"]},  # Filter for failed payments
        "paymentDetails.createdDate": {"$gte": start_date, "$lt": end_date},  # Filter by last 24 hours
        "paymentDetails.amount": {"$exists": True, "$ne": None}  # Ensure amount exists
    }},
    {"$addFields": {"paymentDetails.amount": {"$toDouble": "$paymentDetails.amount"}}},  # Convert amount to double
    {"$group": {"_id": None, "totalAmount": {"$sum": "$paymentDetails.amount"}}}  # Sum the amounts
]

# Execute the aggregation
result_fail = list(collection.aggregate(pipeline_fail))
total_amount_fail = result_fail[0]["totalAmount"] if result_fail else 0

print(f"Total Amount Processed for Failed Payments in the Last 24 Hours: {total_amount_fail}")

### Pending or failed transactions in last 24 hours

query=
  "$and": [
    {
      "$or": [
        { "paymentDetails.createdDate": { "$gte": new Date(new Date().getTime() - 24*60*60*1000) } },
        { "completionDate": { "$gte": new Date(new Date().getTime() - 24*60*60*1000) } }
      ]
    },
    { "paymentDetails.ISOStatus": { "$in": ["RJCT","PDNG", "RCVD"] } }
  ]
}

In [ ]:


# Connect to MongoDB
client = MongoClient(MONGO_URI)
db = client[DATABASE_NAME]
collection = db[COLLECTION_NAME]

# Calculate the last 24 hours range
end_date = datetime.utcnow()
start_date = end_date - timedelta(hours=24)

# Query to count successful payments in the last 24 hours
query_pending_fail = {
    "paymentDetails.ISOStatus": {"$in": ["RJCT","PDNG", "RCVD"]},
    "paymentDetails.createdDate": {"$gte": start_date, "$lt": end_date}
}
pending_fail_payments_count = collection.count_documents(query_pending_fail)
print(f"Number of Pending or failing Payments in the Last 24 Hours: {pending_fail_payments_count}")

# Aggregation pipeline for total amount of successful payments
pipeline_pending_fail = [
    {"$unwind": "$paymentDetails"},  # Flatten the paymentDetails array
    {"$match": {
        "paymentDetails.ISOStatus": {"$in": ["RJCT","PDNG", "RCVD"]},  # Filter for successful payments
        "paymentDetails.createdDate": {"$gte": start_date, "$lt": end_date},  # Filter by last 24 hours
        "paymentDetails.amount": {"$exists": True, "$ne": None}  # Ensure amount exists
    }},
    {"$addFields": {"paymentDetails.amount": {"$toDouble": "$paymentDetails.amount"}}},  # Convert amount to double
    {"$group": {"_id": None, "totalAmount": {"$sum": "$paymentDetails.amount"}}}  # Sum the amounts
]

# Execute the aggregation
result_pending_fail = list(collection.aggregate(pipeline_pending_fail))
total_amount_pending_fail = result_pending_fail[0]["totalAmount"] if result_pending_fail else 0

print(f"Total Amount Processed for pending and failing Payments in the Last 24 Hours: {total_amount_pending_fail}")

### Error records for last 24 hours

Query = {
  "$and": [
    {
      "$or": [
        { "paymentDetails.createdDate": { "$gte": new Date(new Date().getTime() - 24 * 60 * 60 * 1000) } },
        { "completionDate": { "$gte": new Date(new Date().getTime() - 24 * 60 * 60 * 1000) } }
      ]
    },
    {
      "paymentDetails.errorResponse": {
        "$exists": true,
        "$ne": ""
      }
    }
  ]
}

In [ ]:
from pymongo import MongoClient
from datetime import datetime, timedelta


# Connect to MongoDB
client = MongoClient(MONGO_URI)
db = client[DATABASE_NAME]
collection = db[COLLECTION_NAME]

# Calculate the last 24 hours range
end_date = datetime.utcnow()
start_date = end_date - timedelta(hours=24)

# Query to count error records in the last 24 hours
query_error = {
    "$and": [
        {
            "$or": [
                {"paymentDetails.createdDate": {"$gte": start_date, "$lt": end_date}},
                {"completionDate": {"$gte": start_date, "$lt": end_date}}
            ]
        },
        {
            "paymentDetails.errorResponse": {
                "$exists": True,
                "$ne": ""  # Ensure errorResponse is not empty
            }
        }
    ]
}

# Count the error records
error_records_count = collection.count_documents(query_error)
print(f"Number of Error Records in the Last 24 Hours: {error_records_count}")

# Aggregation pipeline for detailed error records
pipeline_error = [
    {"$unwind": "$paymentDetails"},  # Flatten the paymentDetails array
    {"$match": {
        "$and": [
            {
                "$or": [
                    {"paymentDetails.createdDate": {"$gte": start_date, "$lt": end_date}},
                    {"completionDate": {"$gte": start_date, "$lt": end_date}}
                ]
            },
            {
                "paymentDetails.errorResponse": {
                    "$exists": True,
                    "$ne": ""  # Ensure errorResponse is not empty
                }
            }
        ]
    }},
    {"$project": {
        "errorDetails": "$paymentDetails.errorResponse",  # Extract error details
        "createdDate": "$paymentDetails.createdDate",
        "completionDate": "$completionDate"
    }}
]

# Execute the aggregation
error_records = list(collection.aggregate(pipeline_error))

# Print error record details
print(f"Error Records in the Last 24 Hours: {len(error_records)}")
for record in error_records[:5]:  # Print first 5 error records as a sample
    print(record)


## Plot pie chart

In [ ]:
import matplotlib.pyplot as plt

In [ ]:

# Total transactions
total_transactions = (
    successful_payments_count + fail_payments_count + pending_payments_count
)

# Placeholder Image Function
def generate_placeholder_image():
    plt.figure(figsize=(6, 6))
    plt.text(0.5, 0.5, "Pie Chart Here", fontsize=24, ha="center", va="center")
    plt.axis("off")
    placeholder_image = "no_data_placeholder.png"
    plt.savefig(placeholder_image)
    plt.close()
    return placeholder_image

# Generate Pie Chart or Placeholder Image
if total_transactions > 0:
    # Calculate percentages
    successful_percentage = round((successful_payments_count / total_transactions) * 100, 2)
    failed_percentage = round((fail_payments_count / total_transactions) * 100, 2)
    pending_percentage = round((pending_payments_count / total_transactions) * 100, 2)

    # Generate pie chart
    labels = ["Successful", "Failed", "Pending"]
    sizes = [successful_percentage, failed_percentage, pending_percentage]
    colors = ["green", "red", "orange"]
    plt.figure(figsize=(6, 6))
    plt.pie(sizes, labels=labels, autopct="%1.1f%%", colors=colors, startangle=140)
    plt.title("Payment Distribution in Last 24 Hours")
    chart_image = "payment_distribution_pie_chart.png"
    plt.savefig(chart_image)
    plt.close()
else:
    # Generate placeholder image
    chart_image = generate_placeholder_image()

In [ ]:
def generate_bar_chart(success, fail, pending):
    labels = ["Successful", "Failed", "Pending"]
    counts = [success, fail, pending]
    
    if sum(counts) > 0:
        plt.figure(figsize=(8, 5))
        plt.bar(labels, counts, color=["green", "red", "orange"])
        plt.xlabel("Payment Status")
        plt.ylabel("Count")
        plt.title("Payment Activities in Last 24 Hours")
        bar_chart_image = "payment_distribution_bar_chart.png"
        plt.savefig(bar_chart_image)
        plt.close()
        return bar_chart_image
    else:
        print("Bar chart here")
        return None


In [ ]:
from fpdf import FPDF
from requests.auth import HTTPBasicAuth
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

In [ ]:
# Generate Bar Chart or Placeholder Image
def generate_bar_chart():
    if total_transactions > 0:
        labels = ["Successful", "Failed", "Pending"]
        counts = [successful_payments_count, fail_payments_count, pending_payments_count]
        colors = ["green", "red", "orange"]
        
        plt.figure(figsize=(8, 6))
        plt.bar(labels, counts, color=colors)
        plt.xlabel("Payment Status")
        plt.ylabel("Count")
        plt.title("Payment Status in Last 24 Hours")
        
        bar_chart_image = "payment_status_bar_chart.png"
        plt.savefig(bar_chart_image)
        plt.close()
        return bar_chart_image
    else:
        plt.figure(figsize=(8, 6))
        plt.text(0.5, 0.5, "Bar Chart Here", fontsize=24, ha="center", va="center")
        plt.axis("off")
        placeholder_bar_chart = "bar_chart_placeholder.png"
        plt.savefig(placeholder_bar_chart)
        plt.close()
        return placeholder_bar_chart

# Check metrics and generate bar chart
if successful_payments_count == 0 and fail_payments_count == 0 and pending_payments_count == 0:
    bar_chart_image = generate_bar_chart()
else:
    bar_chart_image = generate_bar_chart()

# Add Bar Chart to PDF
pdf.cell(0, 10, "Payment Status Bar Chart", ln=True)
pdf.image(bar_chart_image, x=60, w=90)
pdf.ln(10)

In [ ]:
# Generate a placeholder pie chart when no transactions exist
def generate_placeholder_image():
    plt.figure(figsize=(6, 6))
    plt.text(0.5, 0.5, "Pie Chart Here", fontsize=24, ha="center", va="center")
    plt.axis("off")
    placeholder_image = "payment_distribution_pie_chart.png"
    plt.savefig(placeholder_image)
    plt.close()
    return placeholder_image

# Generate a real pie chart when transactions exist
def generate_real_pie_chart(success, fail, pending):
    labels = ["Successful", "Failed", "Pending"]
    sizes = [success, fail, pending]
    colors = ["green", "red", "orange"]
    plt.figure(figsize=(6, 6))
    plt.pie(sizes, labels=labels, autopct="%1.1f%%", colors=colors, startangle=140)
    plt.title("Payment Distribution in Last 24 Hours")
    pie_chart_image = "payment_distribution_pie_chart.png"
    plt.savefig(pie_chart_image)
    plt.close()
    return pie_chart_image

# Check metrics and generate pie chart
if successful_payments_count == 0 and fail_payments_count == 0 and pending_payments_count == 0:
    chart_image = generate_placeholder_image()
else:
    chart_image = generate_real_pie_chart(successful_payments_count, fail_payments_count, pending_payments_count)

# Generate PDF Report
class PDF(FPDF):
    def header(self):
        # Add logo (top-left)
        self.image("starfish digital logo.png", 10, 8, 30)
        self.set_font("Arial", "B", 16)
        self.cell(0, 10, "Daily Client Report", align="C", ln=True)
        self.ln(5)
        self.set_font("Arial", "I", 12)
        self.cell(0, 10, "Client Name: Traveloka", align="C", ln=True)
        self.ln(10)

    def footer(self):
        self.set_y(-30)
        self.image("footer starfish.png", 10, self.get_y(), 190)
        

    def add_section(self, title, explanation, metric=None):
        self.set_font("Arial", "B", 12)
        self.cell(0, 10, title, ln=True)
        self.set_font("Arial", size=11)
        self.multi_cell(0, 8, explanation)
        if metric is not None:
            self.cell(0, 10, f"   Value: {metric}", ln=True)
        self.ln(5)

# Create PDF
pdf = PDF()
pdf.add_page()

# Date and Time
pdf.set_font("Arial", size=12)
pdf.cell(0, 10, f"Date and Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", ln=True)
pdf.ln(5)

# Pie Chart Section
pdf.cell(0, 10, "Payment Distribution Chart", ln=True)
pdf.image(chart_image, x=60, w=90)
pdf.ln(10)

# Sections
pdf.add_section("Successful Payments", 
                "The total number of payments processed successfully over the past 24 hours. This shows how effectively the platform is handling your transactions.", 
                f"{successful_payments_count}")

pdf.add_section("Pending Payments", 
                "The number of payments currently in progress or awaiting processing. This helps you track transactions that may require further attention.", 
                f"{pending_payments_count}")

pdf.add_section("Failed Payments", 
                "The count of payments that could not be processed due to issues such as incorrect details or insufficient funds. These insights help identify and address any recurring errors.", 
                f"{fail_payments_count}")

pdf.add_section("Errors or Exception Summary",
                "A brief overview of significant issues or errors encountered, along with updates on resolutions. This ensures transparency and proactive communication about any incidents."
                f"{error_records_count}")
                 

# Support Section
pdf.set_font("Arial", "B", 12)
pdf.cell(0, 10, "Support and Contact", ln=True)
pdf.set_font("Arial", size=11)
pdf.multi_cell(0, 8, "For any urgent matters or support requests, please contact our team promptly via the email address provided below:")
pdf.cell(0, 10, "Support Email: support@starfish.digital", ln=True)
pdf.ln(5)

# Save the PDF
pdf_file = "Professional_Daily_Client_Report_With_Placeholder_2.pdf"
pdf.output(pdf_file)
print(f"PDF Report generated and saved as {pdf_file}.")


In [ ]:


# Generate a placeholder pie chart when no transactions exist
def generate_placeholder_image():
    plt.figure(figsize=(6, 6))
    plt.text(0.5, 0.5, "Pie Chart Here", fontsize=24, ha="center", va="center")
    plt.axis("off")
    placeholder_image = "payment_distribution_pie_chart.png"
    plt.savefig(placeholder_image)
    plt.close()
    return placeholder_image

# Generate a real pie chart when transactions exist
def generate_real_pie_chart(success, fail, pending):
    labels = ["Successful", "Failed", "Pending"]
    sizes = [success, fail, pending]
    colors = ["green", "red", "orange"]
    plt.figure(figsize=(6, 6))
    plt.pie(sizes, labels=labels, autopct="%1.1f%%", colors=colors, startangle=140)
    plt.title("Payment Distribution in Last 24 Hours")
    pie_chart_image = "payment_distribution_pie_chart.png"
    plt.savefig(pie_chart_image)
    plt.close()
    return pie_chart_image

# Check metrics and generate pie chart
if successful_payments_count == 0 and fail_payments_count == 0 and pending_payments_count == 0:
    chart_image = generate_placeholder_image()
else:
    chart_image = generate_real_pie_chart(successful_payments_count, fail_payments_count, pending_payments_count)

# Generate PDF Report
class PDF(FPDF):
    def header(self):
        # Add logo (top-left)
        self.image("starfish digital logo.png", 10, 8, 30)
        self.set_font("Arial", "B", 16)
        self.cell(0, 10, "Daily Client Report", align="C", ln=True)
        self.ln(5)
        self.set_font("Arial", "I", 12)
        self.cell(0, 10, "Client Name: Traveloka", align="C", ln=True)
        self.ln(10)

    def footer(self):
        self.set_y(-30)
        self.image("footer starfish.png", 10, self.get_y(), 190)
        

    def add_section(self, title, explanation, metric=None):
        self.set_font("Arial", "B", 12)
        self.cell(0, 10, title, ln=True)
        self.set_font("Arial", size=11)
        self.multi_cell(0, 8, explanation)
        if metric is not None:
            self.cell(0, 10, f"   Value: {metric}", ln=True)
        self.ln(5)

# Create PDF
pdf = PDF()
pdf.add_page()

# Date and Time
pdf.set_font("Arial", size=12)
pdf.cell(0, 10, f"Date and Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", ln=True)
pdf.ln(5)

# Pie Chart Section
pdf.cell(0, 10, "Payment Distribution Chart", ln=True)
pdf.image(chart_image, x=60, w=90)
pdf.ln(10)

# Sections
pdf.add_section("Successful Payments", 
                "The total number of payments processed successfully over the past 24 hours. This shows how effectively the platform is handling your transactions.", 
                f"{successful_payments_count}")

pdf.add_section("Pending Payments", 
                "The number of payments currently in progress or awaiting processing. This helps you track transactions that may require further attention.", 
                f"{pending_payments_count}")

pdf.add_section("Failed Payments", 
                "The count of payments that could not be processed due to issues such as incorrect details or insufficient funds. These insights help identify and address any recurring errors.", 
                f"{fail_payments_count}")

pdf.add_section("Errors or Exception Summary",
                "A brief overview of significant issues or errors encountered, along with updates on resolutions. This ensures transparency and proactive communication about any incidents."
                f"{error_records_count}")
                 

# Support Section
pdf.set_font("Arial", "B", 12)
pdf.cell(0, 10, "Support and Contact", ln=True)
pdf.set_font("Arial", size=11)
pdf.multi_cell(0, 8, "For any urgent matters or support requests, please contact our team promptly via the email address provided below:")
pdf.cell(0, 10, "Support Email: support@starfish.digital", ln=True)
pdf.ln(5)

# Save the PDF
pdf_file = "Professional_Daily_Client_Report_With_Placeholder.pdf"
pdf.output(pdf_file)
print(f"PDF Report generated and saved as {pdf_file}.")
